#### MongoDB에 데이터 insert

필요 라이브러리 사전 설치

In [ ]:
#%pip install wheel
#%pip install pipwin
#%pip install geopandas

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyogrio-0.10.0-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
Using cached pyogrio-0.10.0-cp311-cp311-win_amd64.whl (16.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import geopandas as gpd
import pandas as pd
from fiona.crs import from_epsg
from shapely.geometry import Point
from pymongo import MongoClient

In [26]:
#os.chdir('../')
os.getcwd()

'c:\\Users\\11\\Desktop\\LCG\\Programming\\project\\wedrive'

In [27]:
# 디렉터리 경로
path_directory=os.getcwd()+'/data/test_dir/'
# 센서스 데이터
location_label_path=os.getcwd()+'/data/행정구역구분/BND_SIGUNGU_PG.shp'

# 데이터프레임 칼럼 지정
column_name=['uuid','time_begin','time_end','origin_lat','origin_lng','destination_lat','destination_lng','distance','heart','rank']

In [28]:
# 행정동 데이터 좌표계 변환
location_label=gpd.read_file(location_label_path, encoding='euc-kr')
location_label.to_crs(epsg=4326, inplace=True)
location_label

,BASE_DATE,SIGUNGU_NM,SIGUNGU_CD,geometry
0,20230701,종로구,11010,"POLYGON ((126.98 37.631, 126.97 37.63, 126.97 ..."
1,20230701,중구,11020,"POLYGON ((127.02 37.572, 127.02 37.572, 127.02..."
2,20230701,용산구,11030,"POLYGON ((126.97 37.555, 126.97 37.555, 126.97..."
3,20230701,성동구,11040,"POLYGON ((127.04 37.573, 127.04 37.573, 127.04..."
4,20230701,광진구,11050,"POLYGON ((127.1 37.572, 127.1 37.572, 127.1 37..."
...,...,...,...,...
245,20230701,함양군,38580,"POLYGON ((127.7 35.758, 127.7 35.758, 127.7 35..."
246,20230701,거창군,38590,"POLYGON ((127.88 35.906, 127.88 35.905, 127.88..."
247,20230701,합천군,38600,"POLYGON ((128.1 35.831, 128.1 35.83, 128.1 35...."
248,20230701,제주시,39010,"MULTIPOLYGON (((126.17 33.283, 126.17 33.283, ..."


In [29]:
# raw data에서 칼럼 추가하기
def Make_column(csv_file):
    raw_data=pd.read_csv(csv_file)
    data=raw_data.values.tolist()
    col_data=pd.DataFrame(data, columns=column_name)
    return col_data

# 데이터프레임에 destination_loc 칼럼 추가
def Make_column_destination_loc(csv_file):
    csv_file=Make_column(csv_file)

    #종료 위치 Point 객체
    end_geometry_Point=gpd.points_from_xy(csv_file['destination_lng'], csv_file['destination_lat'])
    coulumns = ['destination_lat', 'destination_lng']
    destination_gdf_data = gpd.GeoDataFrame(csv_file[coulumns], geometry=end_geometry_Point, crs=from_epsg(4326))
    destination_gdf_data['end_geometry'] = destination_gdf_data.apply(lambda row : Point([row['destination_lng'], row['destination_lat']]), axis=1)
    
    # Point 객체에 지역을 매핑시킨 데이터프레임 생성
    df=gpd.sjoin(destination_gdf_data, location_label)
    df.drop(columns=['index_right'], axis=1, inplace=True)
    #최종 데이터 프레임 생성
    csv_file['destination_area']=df['SIGUNGU_NM']
    print('done1')
    return csv_file


In [18]:
# 테스트 b1734fe12445429483e03a242818091d.csv
# 테스트2) 609d7d0a71bd4d68aad1ed698b174372
filename=os.getcwd()+'/data/od_uuid/2024/609d7d0a71bd4d68aad1ed698b174372.csv'
test_dataframe=Make_column_destination_loc(filename)
test_dataframe

C:\Users\11\AppData\Local\Temp\ipykernel_11064\3543792439.py:4: FionaDeprecationWarning: This function will be removed in version 2.0. Please use CRS.from_epsg() instead.
  test_dataframe=Make_column_destination_loc(filename)


,uuid,time_begin,time_end,origin_lat,origin_lng,destination_lat,destination_lng,distance,heart,rank,destination_area
0,609d7d0a71bd4d68aad1ed698b174372,2024-01-10 20:03:27,2024-01-10 20:19:48,37.332608,127.097779,37.294402,127.066992,5801,6,S,수원시 영통구
1,609d7d0a71bd4d68aad1ed698b174372,2024-01-11 09:24:33,2024-01-11 09:31:00,37.313356,127.080340,37.337945,127.102796,3654,3,S,용인시 수지구
2,609d7d0a71bd4d68aad1ed698b174372,2024-01-16 19:50:27,2024-01-16 20:10:31,37.325472,127.090802,37.294271,127.066643,4869,5,S,수원시 영통구
3,609d7d0a71bd4d68aad1ed698b174372,2024-01-17 09:37:41,2024-01-17 09:45:07,37.313521,127.080499,37.338089,127.102762,3581,3,S,용인시 수지구
4,609d7d0a71bd4d68aad1ed698b174372,2024-01-18 09:23:13,2024-01-18 09:32:45,37.297933,127.069321,37.338281,127.102675,5536,5,S,용인시 수지구
5,609d7d0a71bd4d68aad1ed698b174372,2024-01-23 19:55:23,2024-01-24 00:01:38,37.340328,127.095909,37.294567,127.066871,8095,8,A,수원시 영통구


In [30]:
client = MongoClient('10.255.92.37:27017',
          username='netdb',
          password='netdb3230',
          authSource='admin',
          authMechanism='SCRAM-SHA-256')

db = client.get_database('Wedrive2024')
cl = db.get_collection('cg_test_collection')

# 데이터프레임에서 데이터를 추출하여 dictionary 형태로 변경한 후, MongoDB에 데이터 삽입
def dataframe_to_dict(dataframe):
    cl.insert_many(dataframe.to_dict('records'))
    print('done2')

In [24]:
# 디렉터리 내 파일 모두 데이터베이스에 insert
def insert_to_database(dir_path):
    for file in os.listdir(dir_path):
        if file.endswith(".csv"):
            file_path=os.path.join(dir_path, file)
            print(file_path)
            test=Make_column_destination_loc(file_path)
            #dataframe_to_dict(Make_column_destination_loc(file_path))
            test